In [ ]:
import pandas as pd
import numpy as np

In [ ]:
perfomance_splits = [1,2,4,10]

In [ ]:
def filterSplits(df):
    return df[df['dataset'].str.contains('_1_of_')]

In [ ]:
performances_df = pd.concat([filterSplits(pd.read_csv(f'../performances/splitted/1_of_{split}.csv')) for split in perfomance_splits], ignore_index=True)

In [ ]:
def reorder(row, columns):
    sorted_index = row.loc[columns].sort_values(ascending=False).index
    new_columns = [f'v{i}' for i in range(1, len(sorted_index) + 1)]
    return pd.Series(sorted_index.values, index=new_columns)

In [ ]:
performance_df_sorted = performances_df.apply(reorder, axis=1, columns=['RF','LR','GB','ADA'])
performance_df_sorted = pd.concat([performances_df[['dataset']], performance_df_sorted], axis=1)

In [ ]:
concatenated_filtered_sorted_performances = performance_df_sorted

In [ ]:
concatenated_filtered_metafeatures = pd.concat(
    [
        pd.concat(
            [pd.read_csv(f'../metafeatures/splits/labels_1_of_{split}.csv'),
            pd.read_csv(f'../metafeatures/splits/1_of_{split}.csv')], axis=1) 
            for split in perfomance_splits
    ], ignore_index=True)
concatenated_filtered_metafeatures

In [ ]:
performances = concatenated_filtered_sorted_performances[['dataset','v1']]

In [ ]:
metafeatures = concatenated_filtered_metafeatures

In [ ]:
unique_counts = metafeatures.nunique()
columns_to_drop = unique_counts[unique_counts == 1].index
metafeatures = metafeatures.drop(columns_to_drop, axis=1)
metafeatures = metafeatures.dropna(axis=1)

In [ ]:
inf_columns = metafeatures.columns[metafeatures.isin([float('inf')]).any()]
metafeatures = metafeatures.drop(columns=inf_columns)


In [ ]:
metafeatures.set_index("dataset", inplace=True)
performances.set_index("dataset", inplace=True)
metadf = metafeatures.join(performances)
metadf.reset_index(inplace=True)

In [ ]:
metadf = metadf.drop(['dataset'],axis=1)

In [ ]:
metadf.rename(columns={'v1': 'algorithm'}, inplace=True)

In [ ]:
import optunaAlgorithms as opta

In [ ]:
optunadAlgorithms = [
    #{'name':'ADA','classifier':opta.ada},
    #{'name':'GBM','classifier':opta.gbm},
    #{'name':'KNN','classifier':opta.knn},
    #{'name':'LR','classifier':opta.lr},
    #{'name':'NB','classifier':opta.nb},
    {'name':'RF','classifier':opta.rf},
    #{'name':'SVM','classifier':opta.svm}
]

In [ ]:
X = metadf.drop('algorithm',axis=1)
y = metadf['algorithm']

In [ ]:
metadf.to_csv("cleaned_metadf.csv",index=False)

In [ ]:
performances = []
for algorithm in optunadAlgorithms:
    try:
        performances.append({**algorithm["classifier"].optimize(X,y,1000),'algorithm':algorithm['name']})
    except Exception as e:
        print(f"{algorithm['name']} broke !")
        print("why? ", e)

In [ ]:
import json

In [ ]:
perfomances_json = json.dumps(performances, indent=2)

In [ ]:
with open('performances.json', 'w') as json_file:
    json_file.write(perfomances_json)

In [ ]:
[f"{performance['algorithm']}: {performance['accuracy']}" for performance in performances]